In [ ]:
import os
import cv2
import csv
import torchvision.transforms as transforms
import numpy as np
from skimage.metrics import structural_similarity as ssim_skimage
from sewar import ssim as ssim_sewar
from SSIM_PIL import compare_ssim as ssim_pil
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr_skimage
from sewar.full_ref import psnr as psnr_sewar
import lpips
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
def load_images(reference_dir, distorted_dir):
    reference_images = []
    distorted_images = [[None for _ in range(24)] for _ in range(25)]

    # Load reference images: I01.BMP to I25.BMP
    for i in range(1, 26):
        filename = f"I{i:02}.BMP"
        path = os.path.join(reference_dir, filename)
        img = cv2.imread(path)
        reference_images.append(img)

    # Load distorted images: i01_01_5.bmp to i25_24_5.bmp
    for x in range(1, 26):      # x = 01 to 25
        for y in range(1, 25):  # y = 01 to 24
            filename = f"i{x:02}_{y:02}_5.bmp"
            path = os.path.join(distorted_dir, filename)
            img = cv2.imread(path)
            distorted_images[x - 1][y - 1] = img

    return reference_images, distorted_images

In [ ]:
def compute_ssim_metrics(refs, dists):
    results = []

    for i in range(25):  # image ID 0–24
        for j in range(24):  # distortion ID 0–23
            print(f"{i} {j}")
            ref_cv = refs[i]
            dist_cv = dists[i][j]

            # Convert to RGB for skimage and sewar
            ref_rgb = cv2.cvtColor(ref_cv, cv2.COLOR_BGR2RGB)
            dist_rgb = cv2.cvtColor(dist_cv, cv2.COLOR_BGR2RGB)

            # Convert to PIL for SSIM_PIL
            ref_pil = Image.fromarray(ref_rgb)
            dist_pil = Image.fromarray(dist_rgb)

            # SSIM with skimage
            ssim1, _ = ssim_skimage(ref_rgb, dist_rgb, full=True, channel_axis=2)

            # SSIM with sewar
            ssim2 = ssim_sewar(ref_rgb, dist_rgb)[0]

            # SSIM with SSIM_PIL
            ssim3 = ssim_pil(ref_pil, dist_pil)

            results.append([
                f"{i+1:02}", f"{j+1:02}", round(ssim1, 6), round(ssim2, 6), round(ssim3, 6)
            ])
    return results

In [ ]:
def psnr_manual(img1, img2, max_pixel=255.0):
    mse = np.mean((img1 - img2) ** 2)
    return float('inf') if mse == 0 else 20 * np.log10(max_pixel / np.sqrt(mse))

In [ ]:
def compute_psnr_metrics(refs, dists):
    results = []

    for i in range(25):  # image ID 0–24
        for j in range(24):  # distortion ID 0–23
            print(f"{i} {j}")
            ref_cv = refs[i]
            dist_cv = dists[i][j]

            psnr_ski = psnr_skimage(ref_cv, dist_cv, data_range=255)
            psnr_sew = psnr_sewar(ref_cv, dist_cv)
            psnr_man = psnr_manual(ref_cv, dist_cv)

            results.append([
                f"{i+1:02}", f"{j+1:02}", round(psnr_ski, 6), round(psnr_sew, 6), round(psnr_man, 6)
            ])
    return results

In [ ]:
def transform(img):
    # If the image is a numpy array, convert it to PIL Image
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    return transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])(img)

In [ ]:
def compute_lpips_metrics(refs, dists):
    results = []

    lpips_vgg = lpips.LPIPS(net='vgg')
    lpips_alex = lpips.LPIPS(net='alex')
    lpips_squeeze = lpips.LPIPS(net='squeeze')

    for i in range(25):  # image ID 0–24
        for j in range(24):  # distortion ID 0–23
            print(f"{i} {j}")
            ref_cv = refs[i]
            dist_cv = dists[i][j]

            ref_tensor = transform(ref_cv).unsqueeze(0)
            dist_tensor = transform(dist_cv).unsqueeze(0)

            lpips_vgg_value = lpips_vgg(ref_tensor, dist_tensor).item()
            lpips_alex_value = lpips_alex(ref_tensor, dist_tensor).item()
            lpips_squeeze_value = lpips_squeeze(ref_tensor, dist_tensor).item()

            results.append([
                f"{i+1:02}", f"{j+1:02}", round(lpips_vgg_value, 6), round(lpips_alex_value, 6), round(lpips_squeeze_value, 6)
            ])
    return results

In [ ]:
def save_to_csv(results, filename, col1, col2, col3):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image_id', 'distortion_id', col1, col2, col3])
        writer.writerows(results)

In [ ]:
ref_dir = "reference_images"
dist_dir = "distorted_images"

In [ ]:
refs, dists = load_images(ref_dir, dist_dir)
ssim_results = compute_ssim_metrics(refs, dists)
psnr_results = compute_psnr_metrics(refs, dists)
lpip_results = compute_lpips_metrics(refs, dists)
save_to_csv(lpip_results, "lpip_results.csv", 'lpip_vgg', 'lpip_alex', 'lpip_squeeze')
save_to_csv(psnr_results, "psnr_results.csv", 'psnr_skimage', 'psnr_sewar', 'psnr_manual')
save_to_csv(ssim_results, "ssim_results.csv", 'ssim_skimage', 'ssim_sewar', 'ssim_pil')

In [ ]:
def create_charts(method, column1, column2, column3):
    df = pd.read_csv(f"{method}_results.csv")
    # Convert image_id and distortion_id to integers for sorting
    df["image_id"] = df["image_id"].astype(int)
    df["distortion_id"] = df["distortion_id"].astype(int)

    # Create output folder for charts
    os.makedirs(f"charts_{method}", exist_ok=True)

    # Plot for each image_id
    for image_id in sorted(df["image_id"].unique()):
        image_df = df[df["image_id"] == image_id].sort_values("distortion_id")

        x = image_df["distortion_id"]
        width = 0.25

        plt.figure(figsize=(14, 6))
        plt.bar(x - width, image_df[column1], width=width, label="vgg", align="center")
        plt.bar(x, image_df[column2], width=width, label="alex", align="center")
        plt.bar(x + width, image_df[column3], width=width, label="squeeze", align="center")

        plt.xlabel("Distortion ID")
        plt.ylabel(f"{method} Value")
        plt.title(f"{method} Comparison for Image {image_id:02}")
        plt.xticks(ticks=x)
        plt.ylim(0, 1)
        plt.legend()
        plt.tight_layout()

        # Save chart to file
        plt.savefig(f"charts_{method}/image_{image_id:02}_{method}_bar_chart.png")
        plt.close()

In [2]:
create_charts("lpip", "psnr_skimage", "psnr_sewar", "psnr_manual")
create_charts("ssim", "ssim_skimage", "ssim_sewar", "ssim_pil")
create_charts("lpip", "lpip_vgg", "lpip_alex", "lpip_squeeze")

NameError: name 'create_charts' is not defined